In [1]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib
import numpy as np

In [2]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [3]:
path = r"New_Testing_Protocol_Generalized.csv"
chem_path = r"Chemical Database.csv"

In [4]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [5]:
# the goal is to keep the concentration df the same format as all others, hence if you want to do other method you can 
# just need to slightly modify functions. 

In [6]:
# ideally build a function that will pull a component name from a column this would lead to not needing anything provided in the plan if importing!
# but potential issue when calulating volumes say if you need a stock density (well no cause thats shoudl be stock order specific)
# so everything should be order specfic if inputting from plan and using order from there does that make snes

In [7]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan, unity_filter=True) # for now lets work wit this caase and go back and edit
amounts_added = CreateSamples_renewed.determine_component_amounts(concentration_df, plan)
complete_amounts_added = CreateSamples_renewed.calculate_component_amount_missing(amounts_added, plan)
complete_amounts_added_zeroed = CreateSamples_renewed.nan_amounts(complete_amounts_added)

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated component volumes from a component mass using dppc density of nan g/mL
You calculated component volumes from a component mass using dspepeg2000 density of nan g/mL
You calculated component volumes from a component mass using pfh density of 1.691 g/mL
You calculated component volumes from a component mass using ethanol density of 0.7893 g/mL
You calculated component volumes from a component mass using water density of 1.0 g/mL


In [8]:
def stock_dictionary(stock_name, stock_unit, stock_value):
    """The stock name is required to be in the form 'solute n-solvent-stock' where the entry prior to the keyword stock are solvent
    and anything prior to that is assumed a solute. Will return a dictionary of the solvent and solute while pulling information from """
    stock_components = stock_name.split('-')
    stock_solutes = stock_components[:-2] # will always be a list
    stock_solvent = stock_components[-2]
    
    return {'solutes': stock_solutes, 'solvents':stock_solvent, 'unit': stock_unit, 'value': stock_value}

def match_component_to_stock(component_name, stock_names_dict): # how will this deal with other names which contain or multiple instances 
    for key in stock_names_dict.keys():
        if component_name in key:
            return stock_names_dict[key]
        else:
            return AssertionError("Component not present in any stock.")
        
def identify_common_solvents(stocks_dict):
    solvents = [stock_info['solvents'] for stock_name, stock_info in stock_dict.items()]
    common_solvents = list(set([x for x in solvents if solvents.count(x) > 1]))
    for stock_name, stock_info in stock_dict.items(): # make into identify_stocks_solvent_solutes()
        solutes = stock_info['solutes']
        solvents = stock_info['solvents']
        
        for common_solvent in common_solvents:
            if len(solutes) == 0 and common_solvent==solvents:
                stock_dict[stock_name]['Common Solvent'] = 'Pure'
            elif len(solutes) > 0 and common_solvent == solvents:
                stock_dict[stock_name]['Common Solvent'] = 'Mixture'
            else:     
                stock_dict[stock_name]['Common Solvent'] = 'None'
    return stock_dict

def isolate_common_column(df, common_string):
    cols = df.columns
    common_string_cols = [col for col in cols if common_string in col]
    final_df = df.copy()[common_string_cols]
    return final_df

In [9]:
stock_names = plan['Stock Names'] 
stock_units = plan['Stock Concentration Units']
stock_concs = plan['Stock Concentrations']
stock_dict = {stock_name:stock_dictionary(stock_name, stock_unit, stock_conc) for stock_name, stock_unit, stock_conc in zip(stock_names, stock_units, stock_concs)}
stock_dict = identify_common_solvents(stock_dict)

# this failed, just pick how you want to either mass or volume and then do it from there 

In [23]:
components_df = complete_amounts_added_zeroed
component_masses = isolate_common_column(components_df, 'mass')
component_volumes = isolate_common_column(components_df, 'volume')

# so based on what you total_sample_unit is and what you orignially calculated_proceed differently

for col_name in component_masses:
    component_mass = component_masses[col_name]
    component_name = CreateSamples_renewed.identify_component_name(col_name)
    component_unit = CreateSamples_renewed.identify_unit(col_name)
    stock_info = match_component_to_stock(component_name, stock_dict)
    print(component_name, stock_dict.keys())

dppc dict_keys(['dppc-ethanol-stock', 'dspepeg2000-ethanol-stock', 'pfh-ethanol-stock', 'ethanol-stock', 'water-stock'])
dspepeg2000 dict_keys(['dppc-ethanol-stock', 'dspepeg2000-ethanol-stock', 'pfh-ethanol-stock', 'ethanol-stock', 'water-stock'])
pfh dict_keys(['dppc-ethanol-stock', 'dspepeg2000-ethanol-stock', 'pfh-ethanol-stock', 'ethanol-stock', 'water-stock'])
ethanol dict_keys(['dppc-ethanol-stock', 'dspepeg2000-ethanol-stock', 'pfh-ethanol-stock', 'ethanol-stock', 'water-stock'])
water dict_keys(['dppc-ethanol-stock', 'dspepeg2000-ethanol-stock', 'pfh-ethanol-stock', 'ethanol-stock', 'water-stock'])


In [ ]:
def calculate_volumes_from_component_mass(component_mass, component_info, stock_info): 
    
# so create a function for mass based calculation (this will work for sole wtf and volf items)
# grab all masses 
# grab masses component name and unit
# grab the appropiate stock from stock dictionary 
# grab stock value and unit,
# with stock value and unit and component value and unit determeine the correct pathway to calculate and return volume
# Once volumes are caclulated, subtract the component volume if available

    

In [ ]:
def calculate_volumes_from_component_volumes():